In [1]:
import numpy as np

In [2]:
sig_digits = 4
eq_1 = np.random.uniform(low=0, high=13.3, size=(2,)).tolist()
eq_2 =  np.random.uniform(low=0, high=13.3, size=(2,)).tolist()
A = np.array([eq_1, eq_2])
b = np.random.uniform(low=0.5, high=13.3, size=(2,)).tolist()
b = np.array(b)
b = b.T

In [3]:
def Cal_LU(D,g):
    A=np.array((D),dtype=float)
    f=np.array((g),dtype=float)
    n = f.size
    for i in range(0,n-1):     # Loop through the columns of the matrix
        for j in range(i+1,n):     # Loop through rows below diagonal for each column
            if A[i,i] == 0:
                print("Error: Zero on diagonal!")
                print("Need algorithm with pivoting")
                break
            m = A[j,i]/A[i,i]
            A[j,:] = A[j,:] - m*A[i,:]
            f[j]
    return A,f


def Cal_LU_pivot(D,g):
    A=np.array((D),dtype=float)
    f=np.array((g),dtype=float)
    n = len(f)
    for i in range(0,n-1):     # Loop through the columns of the matrix
        
        if np.abs(A[i,i])==0:
            for k in range(i+1,n):
                if np.abs(A[k,i])>np.abs(A[i,i]):
                    A[[i,k]]=A[[k,i]]             # Swaps ith and kth rows to each other
                    f[[i,k]]=f[[k,i]]
                    break
                    
        for j in range(i+1,n):     # Loop through rows below diagonal for each column
            m = A[j,i]/A[i,i]
            A[j,:] = A[j,:] - m*A[i,:]
            f[j] = f[j] - m*f[i]
    return A,f


def Back_Subs(A,f):
    n = f.size
    x = np.zeros(n)             # Initialize the solution vector, x, to zero
    x[n-1] = f[n-1]/A[n-1,n-1]    # Solve for last entry first
    for i in range(n-2,-1,-1):      # Loop from the end to the beginning
        sum_ = 0
        for j in range(i+1,n):        # For known x values, sum and move to rhs
            sum_ = sum_ + A[i,j]*x[j]
        x[i] = (f[i] - sum_)/A[i,i]
    return x

In [4]:
def ge_with_pivot(A, b):
    A, f = Cal_LU_pivot(A, b)
    return Back_Subs(A,f)

In [5]:
def ge_without_pivot(A,b):
    A,f = Cal_LU(A,b)
    return Back_Subs(A,f)

In [6]:
u = ge_without_pivot(A,b)

In [7]:
u_p = ge_with_pivot(A,b)

In [8]:
u

array([1.13691556, 0.29769208])

In [9]:
u_p

array([ 1.64958558, -1.7864234 ])

In [10]:
np.linalg.solve(A,b)

array([ 1.64958558, -1.7864234 ])